# TP1 - Aprendizaje Por Refuerzos
## Integrantes:
- **Azul Noguera** 
- **Paula Jordan** 
- **Rocio Gonzalez**
- **Valentina Pancaldi**

## Enunciado

Implementar un agente de aprendizaje por refuerzo y un ambiente en el cual el agente puede entrenarse. Esto incluye definir la noción de estado del ambiente, que puede no coincidir con el estado del juego.

## Preparación del Entorno

In [122]:
# importamos las clases
from utils import *
from jugador import * # importamos la clase Jugador
from diezmil import * # importamos la clase DiezMil

# importamos funciones de utilidad
from utils import puntaje_y_no_usados, separar, JUGADA_PLANTARSE, JUGADA_TIRAR, JUGADAS_STR

# importamos las librerias necesarias
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import random
from random import randint
import pickle

## Ambiente

In [123]:
class AmbienteDiezMil:
    
    def __init__(self):
        """Definir las variables de instancia de un ambiente.
        ¿Qué es propio de un ambiente de 10.000?
        """
        self.estado = EstadoDiezMil()
        self.turno_terminado = False  # flag que indica si el turno terminó
        self.puntaje_total = 0  # Mantener un registro del puntaje total

    def reset(self):
        """
        Reinicia el estado del ambiente.
        """
        self.estado.reset()
        self.turno_terminado = False
        # Aquí no reseteamos `self.puntaje_total` para mantener el acumulado entre episodios.
        return self.estado.obtener_estado()
    
    def step(self, accion):
        """
        Dada una acción devuelve una recompensa.
        El estado es modificado acorde a la acción y su interacción con el ambiente.
        Podría ser útil devolver si terminó o no el turno.
        
        Args:
            accion: Acción elegida por un agente.
        
        Returns:
            tuple[int, bool]: Una recompensa y un flag que indica si terminó el turno. 
        """
        print(f"\n{'='*50}")
        print(f"Acción Elegida: {accion.upper()}")

        print(f"Estado Antes de la Acción: {self.estado}")
        
        if accion == 'plantarse':
            self.turno_terminado = True
            recompensa = self.estado.puntaje_actual
            self.puntaje_total += recompensa  # Acumular la recompensa al puntaje total
            
        elif accion == 'tirar':
            nuevos_dados = self.estado.lanzar_dados(len(self.estado.dados))
            puntaje, dados_no_usados = puntaje_y_no_usados(nuevos_dados)
            print(f"Tirada de Dados: {nuevos_dados}")
            print(f"Puntaje Calculado: {puntaje}, Dados No Usados: {dados_no_usados}")
            self.estado.actualizar_estado(dados_no_usados, puntaje)
            recompensa = self.estado.puntaje_actual
            self.puntaje_total += recompensa  # Acumular la recompensa al puntaje total
            if recompensa == 0:
                self.turno_terminado = True
        else:
            raise ValueError("Acción no válida.")
        
        print(f"Estado Después de la Acción: {self.estado}")
        print(f"Recompensa Obtenida: {recompensa}")
        print(f"Puntaje Total Acumulado: {self.puntaje_total}")

        # Condición para terminar el juego si el puntaje total es 10,000 o más
        if self.puntaje_total >= 10000:
            self.estado.fin_turno()
            print(f"Juego terminado. Puntaje Total: {self.puntaje_total}")

        print(f"{'='*50}\n")
        
        return self.estado.obtener_estado(), recompensa, self.turno_terminado

## Estado

In [124]:
class EstadoDiezMil:
    def __init__(self):
        """Definir qué hace a un estado de diez mil.
        Recordar que la complejidad del estado repercute en la complejidad de la tabla del agente de q-learning.
        """
        self.dados = []  # Lista de los valores actuales de los dados
        self.puntaje_actual = 0  # Puntaje acumulado en el turno actual
        self.turno_terminado = False  # Flag para indicar si el turno ha terminado

    def reset(self):
        """Reinicia el estado del juego a sus valores iniciales."""
        self.dados = self.lanzar_dados(6)  # Reiniciar los dados
        self.puntaje_actual = 0  # Reiniciar el puntaje acumulado
        self.turno_terminado = False  # Reiniciar el flag de turno terminado

    def lanzar_dados(self, cantidad: int) -> list[int]:
        """Simula el lanzamiento de una cantidad específica de dados.
        Args:
            cantidad (int): La cantidad de dados a lanzar.
        Returns:
            list[int]: Los valores obtenidos en los dados lanzados.
        """
        return [randint(1, 6) for _ in range(cantidad)]

    def obtener_estado(self):
        """Devuelve el estado actual del juego en un formato que el agente pueda utilizar."""
        return (self.dados, self.puntaje_actual, self.turno_terminado)

    def actualizar_estado(self, nuevos_dados: list[int], puntaje: int) -> None:
        """Modifica las variables internas del estado luego de una tirada.
        Args:
            nuevos_dados (list[int]): Dados no usados tras la tirada.
            puntaje (int): Puntaje obtenido en la tirada.
        """
    
        self.dados = nuevos_dados
        self.puntaje_actual += puntaje
        if len(self.dados) == 0:
            # Si no quedan dados no usados, el turno debe terminar
            self.turno_terminado = True
    
        
    
    def fin_turno(self):
        """Modifica el estado al terminar el turno.
        """
        self.turno_terminado = True  # Marca el turno como terminado

    def __str__(self):
        """Representación en texto de EstadoDiezMil.
        Ayuda a tener una versión legible del objeto.

        Returns:
            str: Representación en texto de EstadoDiezMil.
        """
        estado_str = f"Dados: {self.dados}, Puntaje Actual: {self.puntaje_actual}, Turno Terminado: {self.turno_terminado}"
        return estado_str   

## Agente

In [125]:
class AgenteQLearning:
    def __init__(self, ambiente: AmbienteDiezMil, alpha: float, gamma: float, epsilon: float):
        self.ambiente = ambiente
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}

    def elegir_accion(self, estado):
        # Convertir cada parte del estado a una tupla si es necesario
        estado = tuple(tuple(part) if isinstance(part, list) else part for part in estado)
        
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(['plantarse', 'tirar'])
        else:
            if estado in self.q_table:
                return max(self.q_table[estado], key=self.q_table[estado].get)
            else:
                return random.choice(['plantarse', 'tirar'])

    def entrenar(self, episodios: int, verbose: bool = False) -> None:
        for episodio in tqdm(range(episodios)):
            estado = self.ambiente.reset()
            estado = tuple(tuple(part) if isinstance(part, list) else part for part in estado)
            terminado = False

            while not terminado:
                accion = self.elegir_accion(estado)
                nuevo_estado, recompensa, terminado = self.ambiente.step(accion)
                nuevo_estado = tuple(tuple(part) if isinstance(part, list) else part for part in nuevo_estado)

                if estado not in self.q_table:
                    self.q_table[estado] = {'plantarse': 0, 'tirar': 0}
                
                if nuevo_estado not in self.q_table:
                    self.q_table[nuevo_estado] = {'plantarse': 0, 'tirar': 0}

                mejor_q_nuevo_estado = max(self.q_table[nuevo_estado].values())
                self.q_table[estado][accion] += self.alpha * (
                    recompensa + self.gamma * mejor_q_nuevo_estado - self.q_table[estado][accion]
                )

                estado = nuevo_estado

                if verbose:
                    print(f"Paso: {episodio + 1}, Estado: {estado}, Acción: {accion}, Recompensa: {recompensa}")

    def guardar_politica(self, filename: str):
        with open(filename, 'wb') as file:
            pickle.dump(self.q_table, file)

## Entrenamiento

In [126]:
class JugadorEntrenado(Jugador):
    def __init__(self, nombre: str, filename_politica: str):
        self.nombre = nombre
        self.politica = self._leer_politica(filename_politica)
        
    def _leer_politica(self, filename:str, SEP:str=','):
        """Carga una politica entrenada con un agente de RL, que está guardada
        en el archivo filename en un formato conveniente.

        Args:
            filename (str): Nombre/Path del archivo que contiene a una política almacenada. 
        """
        with open(filename, 'rb') as file:
            politica = pickle.load(file)
        return politica
    
    def jugar(self, puntaje_total:int, puntaje_turno:int, dados:list[int]) -> tuple[int,list[int]]:
        """
        Devuelve una jugada y los dados a tirar.
        Args:
            puntaje_total (int): Puntaje total del jugador en la partida.
            puntaje_turno (int): Puntaje en el turno del jugador
            dados (list[int]): Tirada del turno.
        Returns:
            tuple[int,list[int]]: Una jugada y la lista de dados a tirar.
        """

        # Calcular el puntaje actual y los dados no usados
        puntaje, no_usados = puntaje_y_no_usados(dados)

        # Definir el estado como una tupla con los componentes relevantes
        estado = (puntaje_total, puntaje_turno, tuple(dados))

        # Consultar la política para determinar la jugada
        jugada = self.politica.get(estado, JUGADA_PLANTARSE)  # Valor predeterminado si el estado no está en la política

        if jugada == JUGADA_PLANTARSE:
            return (JUGADA_PLANTARSE, [])
        elif jugada == JUGADA_TIRAR:
            return (JUGADA_TIRAR, no_usados)

## Ejecución

In [127]:
ambiente = AmbienteDiezMil()
agente = AgenteQLearning(ambiente, alpha=0.1, gamma=0.9, epsilon=0.2)
agente.entrenar(episodios=100, verbose=True)

100%|██████████| 100/100 [00:00<00:00, 10415.46it/s]


Acción Elegida: PLANTARSE
Estado Antes de la Acción: Dados: [3, 5, 4, 1, 1, 6], Puntaje Actual: 0, Turno Terminado: False
Estado Después de la Acción: Dados: [3, 5, 4, 1, 1, 6], Puntaje Actual: 0, Turno Terminado: False
Recompensa Obtenida: 0
Puntaje Total Acumulado: 0

Paso: 1, Estado: ((3, 5, 4, 1, 1, 6), 0, False), Acción: plantarse, Recompensa: 0

Acción Elegida: TIRAR
Estado Antes de la Acción: Dados: [5, 3, 2, 6, 6, 5], Puntaje Actual: 0, Turno Terminado: False
Tirada de Dados: [3, 6, 5, 6, 6, 4]
Puntaje Calculado: 650, Dados No Usados: [3, 4]
Estado Después de la Acción: Dados: [3, 4], Puntaje Actual: 650, Turno Terminado: False
Recompensa Obtenida: 650
Puntaje Total Acumulado: 650

Paso: 2, Estado: ((3, 4), 650, False), Acción: tirar, Recompensa: 650

Acción Elegida: PLANTARSE
Estado Antes de la Acción: Dados: [3, 4], Puntaje Actual: 650, Turno Terminado: False
Estado Después de la Acción: Dados: [3, 4], Puntaje Actual: 650, Turno Terminado: False
Recompensa Obtenida: 650
Punt

## Evaluación

In [99]:
jugador = JugadorEntrenado("Agente Trained", "politica_qlearning.pkl")
resultado = jugador.jugar(puntaje_total=0, puntaje_turno=0, dados=[1, 2, 3, 4, 5, 6])
print(resultado)

(0, [])


## Guardado de Política

In [100]:
agente.guardar_politica("politica_qlearning.pkl")